In [1]:
from jupyter_plotly_dash import JupyterDash
#from jupyter_dash import JupyterDash
import dash
import dash_leaflet as dl
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import dash_table as dt
from dash.dependencies import Input, Output, State

import os
import numpy as np
import pandas as pd
from pymongo import MongoClient
from bson.json_util import dumps

import base64

#### FIX ME #####
# Changed animal_shelter and AnimalShelter to match CRUD Python module file name and class name
from animalShelter import AnimalShelter


###########################
# Data Manipulation / Model
###########################
# FIX ME change for your username and password and CRUD Python module name. Username and password hard coded in as instructed.
username = "aacuser"
password = "CS-340-T3339"
shelter = AnimalShelter(username=username, password=password)

# class read method must support return of cursor object 
df = pd.DataFrame.from_records(shelter.read({}))


#########################
# Dashboard Layout / View
#########################
app = JupyterDash('SimpleExample')

#FIX ME Add in Grazioso Salvare’s logo. Logo added from Grazioso Salvare supported materials.
image_filename = 'Grazioso Salvare Logo.png' # replace with your own image
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

#FIX ME Place the HTML image tag in the line below into the app.layout code according to your design
#FIX ME Also remember to include a unique identifier such as your name or date
#html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()))
#
table_columns=[
    'name', 'animal_type', 'breed', 'color', 'date_of_birth', 'outcome_type', 'sex_upon_outcome'
]

app.layout = html.Div([
#    html.Div(id='hidden-div', style={'display':'none'}),
    
    html.Div(className="row", style={'display': 'flex'}, children=[       
        #Embed logo here.
        html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()), width='120px'),
        html.Div(className="row", children=[
        html.B(html.H1('SNHU CS-340 Dashboard')),
        html.B("Jessica Megaro"),
        ])
        
    ]),
    html.Hr(),
    html.Div(
        #FIXME Add in code for the interactive filtering options. For example, Radio buttons, drop down, checkboxes, etc.
        #Added radio buttons to select one option at a time.
        dcc.RadioItems(
            options=[
                {'label': 'Water Rescue', 'value': 'water'},
                {'label': 'Mountain Rescue', 'value': 'mountain'},
                {'label': 'Disaster Rescue', 'value': 'disaster'},
                {'label': 'Reset', 'value': 'reset'}
            ],
            labelStyle={'display': 'inline-block'},
            id='filter-type'
        )
    ),
    html.Hr(),
    dt.DataTable(
        id='datatable-id',
        #Columns limited for ease of use to only include necessary data.
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in table_columns
        ],
        data=df.to_dict('records'),
        #FIXME: Set up the features for your interactive data table to make it user-friendly for your client
        #If you completed the Module Six Assignment, you can copy in the code you created here 
        #Pages set to 10 to decrease size of return for ease of use. 
        page_size=10,        
    ),
    html.Div(style={"clear": "both"}),
    html.Br(),
    html.Hr(),
    #This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row',
             style={'display' : 'flex'},
             children=[
                html.Div(
                     id='graph-id',
                     className='col s12 m6',
                     style={'width': '50%', 'height': '50vh'},
                 ),
                 html.Div(
                     id='map-id',
                     className='col s12 m6',
                     style={'width': '50%', 'height': '50vh'},
                 )
             ]
    )
])

#############################################
# Interaction Between Components / Controller
#############################################


@app.callback([Output('datatable-id','data'),
               Output('datatable-id','columns')],
              [Input('filter-type', 'value')])
def update_dashboard(filter_type):
    ### FIX ME Add code to filter interactive data table with MongoDB queries
    #Queries set to reflect specification document requirements as instructed.
    global df
    query = {}
    if filter_type == 'water':
        query = {
            'breed': {'$in': ['Labrador Retriever Mix', 'Chesapeake Bay Retriever', 'Newfoundland']},
            'sex_upon_outcome': 'Intact Female',
            '$and': [
                {'age_upon_outcome_in_weeks': {'$gte': 26}},
                {'age_upon_outcome_in_weeks': {'$lte': 156}}
            ]
        }
    elif filter_type == 'mountain':
        query = {
            'breed': {'$in': ['German Shepherd', 'Alaskan Malamute', 'Old English Sheepdog',
                              'Siberian Husky', 'Rottweiler']},
            'sex_upon_outcome': 'Intact Male',
            '$and': [
                {'age_upon_outcome_in_weeks': {'$gte': 26}},
                {'age_upon_outcome_in_weeks': {'$lte': 156}}
            ]
        }
    elif filter_type == 'disaster':
        query = {
            'breed': {'$in': ['Doberman Pinscher', 'German Shepherd', 'Golden Retriever', 'Bloodhound',
                              'Rottweiler']},
            'sex_upon_outcome': 'Intact Male',
            '$and': [
                {'age_upon_outcome_in_weeks': {'$gte': 20}},
                {'age_upon_outcome_in_weeks': {'$lte': 300}}
            ]
        }

    columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in table_columns]

    df = pd.DataFrame.from_records(shelter.read(query))
    data = df.to_dict('records')            
    return (data,columns)


@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    if selected_columns is not None: #If statement added to eliminate error on startup.
        return [{
            'if': { 'column_id': i },
            'background_color': '#D2F3FF'
        } for i in selected_columns]
    else:
        []

@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])
def update_graphs(viewData):
    ###FIX ME ####
    # add code for chart of your choice (e.g. pie chart) #
    #Pie chart added and updates based on current page being displayed.
    dff = pd.DataFrame.from_dict(viewData)
    breeds = dff.groupby(['breed'])['breed'].count().reset_index(name='count')
    return [
        dcc.Graph(
            figure=px.pie(breeds, values='count', names='breed',
                            color_discrete_sequence=px.colors.sequential.RdBu),
            style={'padding': '0', 'margin': '0'}
        )
    ]

#Creates marker to represent position in the interactive map.
def animal_marker(animal): 
    return dl.Marker(position=[animal[13], animal[14]], children=[
        dl.Tooltip(animal[4]),
        dl.Popup([
            html.H1("Animal Name"),
            html.P(animal[9])
        ])
    ])
                               
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])
def update_map(viewData):
    dff = pd.DataFrame.from_dict(viewData)
    if len(dff.axes[0]) <= 0: # Callback gets called before data is loaded.
        return []
    lat_first = dff.iloc[0, 13]
    lon_first = dff.iloc[0, 14]
    #Creates marker for all items currently displayed.
    children = [animal_marker(row) for (idx, row) in dff.iterrows()]
    children.insert(0, dl.TileLayer())
    return [
        dl.Map(children=children, id="map",
            style={'width': '100%', 'height': '50vh', 'margin': '0', 'padding': '0'},
            center=[30.75, -97.48]
        ),
    ]

app
#app.run_server(debug=True, port=9060)